**Kaggle team name**: Interstellar_Na_Rivas_Yoo 
1. Format: `🐧[BB Team#]-[Your fancy Kaggle group name]-[Students' names]`. 
  1. Eg. `🐧A-Heros-Fleischer,Melnikov`, where `🐧` identifies JHU and `A` identifies the group code in Canvas

Your private LB score must be reproducible with this Colab. Seed all [RNG](https://en.wikipedia.org/wiki/Random_number_generation). Don't exceed runtime quota.

<small><hr style="margin:0;background-color:silver"><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>, ©<a href="https://apps.ep.jhu.edu/course-homepages/3765-605-742-deep-neural-networks" target="_blank">JHU</a> 2021 onwards</font></small>

<hr color=darkblue>

# **2🏆🌌Stellar**

<details><summary><font color=darkblue>More info and Kaggle API instructions</font></summary>

[Kaggle competition URL](https://www.kaggle.com/c/jan-30-22stellar/rules). See competition rules, submission, grading, dataset, and performance metric. The **starter code** below produces a baseline model, which you should beat, while respecting the competition rules. Your code starts after the timer. This is your baseline model. Seed all [RNG](https://en.wikipedia.org/wiki/Random_number_generation) for reproducibility!

**Instructions for enabling Kaggle API in Colab**:
1. Accept competition rules before running [Kaggle API](https://github.com/Kaggle/kaggle-api#api-credentials). [Loading Kaggle dataset example](https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab)
1. In your Kaggle Account, [Create API Token](https://github.com/Kaggle/kaggle-api#api-credentials) and save the resulting **kaggle.json** file to the [root of your Google Drive](https://drive.google.com/drive/u/0/my-drive) 
2. In Colab, open **Files** panel 🗀 (on the left) and click gray folder icon <font color=gray>🖿</font> to mount your Google drive

Your Kaggle/Google Drive credentials are secure; and Colab's kaggle.json only lasts a Colab session.

</details>


In [ ]:
#from google.colab import drive; drive.mount('/content/gdrive')   # OK to enable, if kaggle.json is stored in Google Drive

In [1]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log    # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                                   # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
#!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json > log   # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                           # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                                      # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v jan-30-22stellar                 # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                                  # download competition dataset as a zip file
!unzip -o *.zip >> log                                                # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                               # print public leaderboard

- competition is now set to: jan-30-22stellar
100% 7.58M/7.58M [00:01<00:00, 5.44MB/s]
Using competition: jan-30-22stellar
 teamId  teamName                                           submissionDate       score    
-------  -------------------------------------------------  -------------------  -------  
8767527  🐧5-BabyYodas-NathansonNguyen                       2022-06-20 00:12:15  0.96905  
8103227  JC-OutOfThisWorld-Lee,Brodsky                      2022-02-13 15:48:37  0.96695  
8111730  JA-Superstars-Corson,Givre,Hogge                   2022-02-13 02:28:00  0.96685  
8087531  JE-TBD-Mendola, Xiang                              2022-02-10 19:46:07  0.96665  
8088888  JF-LOGsters-Dinh-Hernandez                         2022-02-13 09:35:24  0.96635  
8094938  JB-TBD-Khan,Natali                                 2022-02-13 00:28:33  0.96555  
8118756  JD-Allstars-Merran,Hu                              2022-02-14 02:36:08  0.96395  
8153712  DP-nuts-Korzun,Kotukhov,Vasyutin                 

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all" 
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('max_columns', 20, 'precision', 2, 'display.max_rows', 4)

CPU times: user 758 ms, sys: 290 ms, total: 1.05 s
Wall time: 808 ms


In [ ]:
df = pd.read_csv('XY_Stellar.csv'); df.head()

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID,Class
0,11.64,21.28,26.28,26.15,24.05,18.87,19.00,8848,5,272,0.84,7740,56824,833,NaN
1,173.09,42.21,22.51,22.83,22.21,19.55,19.96,4156,3,486,0.81,9041,58067,428,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,31.91,1.02,22.09,23.59,19.36,18.56,18.21,8589,1,610,0.39,704,52136,463,NaN
4,4.71,-1.19,21.26,22.35,18.07,17.48,17.15,4485,1,152,0.21,4249,55417,243,NaN


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   alpha     200000 non-null  float64
 1   delta     200000 non-null  float64
 2   u         200000 non-null  float64
 3   g         200000 non-null  float64
 4   r         200000 non-null  float64
 5   i         200000 non-null  float64
 6   z         200000 non-null  float64
 7   run_ID    200000 non-null  int64  
 8   cam_col   200000 non-null  int64  
 9   field_ID  200000 non-null  int64  
 10  redshift  200000 non-null  float64
 11  plate     200000 non-null  int64  
 12  MJD       200000 non-null  int64  
 13  fiber_ID  200000 non-null  int64  
 14  Class     160000 non-null  object 
dtypes: float64(8), int64(6), object(1)
memory usage: 22.9+ MB


In [ ]:
df.describe().T.style.background_gradient(cmap = 'CMRmap')

,count,mean,std,min,25%,50%,75%,max
alpha,200000.00,177.61,98.22,0.00,124.69,178.83,235.21,428.60
delta,200000.00,25.96,21.46,-18.76,5.32,24.91,42.84,97.46
u,200000.00,23.37,2.73,-182.97,21.48,23.39,25.17,169.34
g,200000.00,23.62,3.04,-119.45,21.35,23.65,25.68,111.99
r,200000.00,20.79,2.22,-89.20,19.23,21.08,22.33,117.72
i,200000.00,17.98,1.98,-96.19,16.66,18.14,19.33,124.05
z,200000.00,18.23,1.83,-73.06,16.94,18.41,19.43,135.56
run_ID,200000.00,4805.88,2233.30,109.00,3387.00,4383.00,5795.00,9452.00
cam_col,200000.00,3.51,1.59,1.00,2.00,4.00,5.00,6.00
field_ID,200000.00,192.07,160.48,11.00,82.00,147.00,247.00,1157.00


In [ ]:
vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample
tXY = df.query('Class==Class')                       # slice training sample

# rs = [1 if x > 0 else -1 for x in tXY['redshift']]
# tXY['rs_bin'] = rs

# #df['max_filter'] = df[['u', 'g', 'r', 'i', 'z']].idxmax(axis=1)
# tXY['max_filter'] = tXY[['u', 'g', 'r', 'i', 'z']].idxmax(axis=1)
# tXY['mf'] = maxFilter(tXY)
# tXY = tXY.drop('max_filter', axis=1)
#tX, tY = tXY.drop('Class', axis=1), tXY.Class       # split into training I/O

In [ ]:
original_indices = tXY.index
original_indices

Int64Index([ 40000,  40001,  40002,  40003,  40004,  40005,  40006,  40007,
             40008,  40009,
            ...
            199990, 199991, 199992, 199993, 199994, 199995, 199996, 199997,
            199998, 199999],
           dtype='int64', length=160000)

In [ ]:
reset_ind = tXY.reset_index().index.tolist()
df_reset_ind = tXY.reset_index()
#reset_ind

In [ ]:
from imblearn.under_sampling import RandomUnderSampler, NearMiss 

nm = NearMiss(sampling_strategy='majority')
#X_res, y_res = rus.fit_resample(X, y)
#tX, tY = rus.fit_resample(tXY.drop('Class', axis=1), tXY.Class)#indices are automatically reset
tX, tY = nm.fit_resample(df_reset_ind.drop('Class', axis=1), tXY['Class'])

In [ ]:
tX.shape

(95230, 15)

In [ ]:
tXY = tX.join(tY)

In [ ]:
tXY.head()

,index,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID,Class
0,91306,22.71,12.13,22.92,23.33,21.85,19.61,19.91,9046,3,480,0.45,11167,58382,275,G
1,63359,219.25,49.11,23.82,24.95,20.67,19.45,19.23,3967,2,89,0.19,7495,56718,280,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,44654,257.98,47.10,25.22,26.46,21.88,18.51,18.90,3967,3,181,0.48,8577,57405,822,G
4,63915,163.41,53.19,25.19,24.41,23.47,17.43,18.65,3967,5,86,0.13,7495,56673,661,G


In [ ]:
#tX.sort_values('index', inplace=True)
tXY.sort_values('index', inplace=True)

In [ ]:
tXY.head()

,index,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID,Class
30434,40000,375.95,26.82,24.83,24.05,23.04,18.82,19.38,6710,6,164,2.51e+00,7786,57271,789,Q
30435,40004,230.19,44.68,20.53,20.86,19.90,19.02,19.07,3967,2,171,1.58e+00,8678,57468,969,Q
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27436,40006,256.79,52.15,24.56,23.74,23.20,17.92,18.49,3438,6,164,7.32e-01,6850,56367,324,G
60869,40008,222.71,57.55,30.31,30.03,25.21,16.87,17.79,3029,6,373,-3.00e-04,8418,57324,163,S


In [ ]:
tXY.set_index(['index'], inplace = True, drop = True)

In [ ]:
tXY.head()

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID,Class
index,,,,,,,,,,,,,,,
40000,375.95,26.82,24.83,24.05,23.04,18.82,19.38,6710,6,164,2.51e+00,7786,57271,789,Q
40004,230.19,44.68,20.53,20.86,19.90,19.02,19.07,3967,2,171,1.58e+00,8678,57468,969,Q
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40006,256.79,52.15,24.56,23.74,23.20,17.92,18.49,3438,6,164,7.32e-01,6850,56367,324,G
40008,222.71,57.55,30.31,30.03,25.21,16.87,17.79,3029,6,373,-3.00e-04,8418,57324,163,S


In [ ]:
tXY.index.name = None
tX.head()

,index,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID
0,91306,22.71,12.13,22.92,23.33,21.85,19.61,19.91,9046,3,480,0.45,11167,58382,275
1,63359,219.25,49.11,23.82,24.95,20.67,19.45,19.23,3967,2,89,0.19,7495,56718,280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,44654,257.98,47.10,25.22,26.46,21.88,18.51,18.90,3967,3,181,0.48,8577,57405,822
4,63915,163.41,53.19,25.19,24.41,23.47,17.43,18.65,3967,5,86,0.13,7495,56673,661


In [ ]:
tX, tY = tXY.drop('Class', axis=1), tXY.Class

In [ ]:
tX.columns

Index(['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'run_ID', 'cam_col',
       'field_ID', 'redshift', 'plate', 'MJD', 'fiber_ID'],
      dtype='object')

In [ ]:
tX.head()

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID
40000,375.95,26.82,24.83,24.05,23.04,18.82,19.38,6710,6,164,2.51e+00,7786,57271,789
40004,230.19,44.68,20.53,20.86,19.90,19.02,19.07,3967,2,171,1.58e+00,8678,57468,969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40006,256.79,52.15,24.56,23.74,23.20,17.92,18.49,3438,6,164,7.32e-01,6850,56367,324
40008,222.71,57.55,30.31,30.03,25.21,16.87,17.79,3029,6,373,-3.00e-04,8418,57324,163


In [ ]:
tXY.cam_col.unique()

array([6, 2, 4, 1, 5, 3])

###REMOVE OUTLIERS

In [ ]:
def outliers(df, ft):
  # q1 = df[ft].quantile(.05)
  # q3 = df[ft].quantile(.95)
  q1 = df[ft].quantile(.1)
  q3 = df[ft].quantile(.9)

  iqr = q3 - q1
  
  low_bound = q1 - 1.5 * iqr
  upp_bound = q3 + 1.5 * iqr

  ls = df.index[(df[ft] < low_bound) | (df[ft] > upp_bound)]

  return ls

In [ ]:
index_list = []

for col in tX[['alpha', 'delta', 'redshift']].columns:
  index_list.extend(outliers(tX, col))

In [ ]:
len(index_list)

316

In [ ]:
def remove(df, ls):
  ls = sorted(set(ls))
  df = df.drop(ls)
  return df

In [ ]:
df_cleaned = remove(tXY, index_list)

In [ ]:
df_cleaned.shape

(94914, 15)

In [ ]:
df_cleaned.columns

Index(['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'run_ID', 'cam_col',
       'field_ID', 'redshift', 'plate', 'MJD', 'fiber_ID', 'Class'],
      dtype='object')

###SCALE

In [ ]:
####tX_norm = (df_cleaned[['alpha', 'delta', 'redshift', 'u', 'g', 'r', 'i', 'z']] - df_cleaned[['alpha', 'delta', 'redshift', 'u', 'g', 'r', 'i', 'z']].min())/(df_cleaned[['alpha', 'delta', 'redshift', 'u', 'g', 'r', 'i', 'z']].max() - df_cleaned[['alpha', 'delta', 'redshift', 'u', 'g', 'r', 'i', 'z']].min())

In [ ]:
####tXY_ = tX_norm.join(df_cleaned[['run_ID', 'cam_col', 'field_ID', 'plate', 'MJD', 'fiber_ID', 'Class']])

In [ ]:
####tXY_ = pd.get_dummies(tXY_, columns=['cam_col'])

In [ ]:
####tX, tY = tXY_.drop('Class', axis=1), tXY_.Class        # split into training I/O

In [ ]:
####tX.head()

In [ ]:
#tX, tY = df_cleaned.drop('Class', axis=1), df_cleaned.Class 

<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

Below is a baseline model that produces the result on Kaggle leaderboard (LB).

In [ ]:
# def maxFilter(df):
#   mf = list()
#   for val in df['max_filter']:
#     if val == 'u':
#       mf.append(1)
#     elif val == 'g':
#       mf.append(2)
#     elif val == 'r':
#       mf.append(3)
#     elif val == 'i':
#       mf.append(4)
#     elif val == 'z':
#       mf.append(5)
#   return mf

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder


def final_pipeline(df, df_Y=None):
  # cam_col_cats = [1, 2, 3, 4, 5, 6]
  # ordinal_encoder = OneHotEncoder(categories=[cam_col_cats])
  # df_oe = df[['cam_col']]
  # df_oe_encoded = pd.DataFrame(ordinal_encoder.fit_transform(df_oe).toarray(), index=df_oe.index)
  # df = pd.concat([df, df_oe_encoded], axis=1)

  # cam_col_cat =[ 1, 2, 3, 4, 5, 6]
  # one_hot_encoder = OneHotEncoder(categories=[cam_col_cat])
  # df_ohe = df[['cam_col']]
  # df_ohe_encoded = pd.DataFrame(one_hot_encoder.fit_transform(df_ohe).toarray(), index=df_ohe.index)

  # df = df.join(df_ohe_encoded)

  if type(df_Y)!=type(None):
    df_y = df_Y.loc[[x for x in df.index]]
  
  #df = df.drop(['cam_col'], axis=1)
  df = df.drop(['run_ID'],axis=1)
  df = df.drop(['field_ID'],axis=1)
  df = df.drop(['fiber_ID'],axis=1)
  df = df.drop(['plate'],axis=1)
  df = df.drop(['MJD'],axis=1)

  #df = (df-df.mean())/df.std()
  df['max'] = df[['u', 'g', 'r', 'i', 'z']].apply(np.max, axis = 1)
  #df['min'] = df[['u', 'g', 'r', 'i', 'z']].apply(np.min, axis = 1)
  df['avg'] = df[['u', 'g', 'r', 'i', 'z']].apply(np.average, axis = 1)
  #df['std'] = df[['u', 'g', 'r', 'i', 'z']].apply(np.std, axis = 1)
  df['var'] = df[['u', 'g', 'r', 'i', 'z']].apply(np.var, axis = 1)

  # rs = [1 if x > 0 else -1 for x in df['redshift']]
  # df['rs_bin'] = rs

  # df['max_filter'] = df[['u', 'g', 'r', 'i', 'z']].idxmax(axis=1)
  # df['mf'] = maxFilter(df)
  # df = df.drop('max_filter', axis=1)

  #df = df.drop('redshift', axis=1)

  df = df.drop(['u', 'g', 'r', 'i', 'z'], axis=1)

  if type(df_Y)!=type(None):
    return df, df_y
  else:
    return df

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict

tX0, tY0 = tX.iloc[:20000, :], tY[:20000]   # subsample for model selection experiments
vX0, vY0 = tX.iloc[-20000:, :], tY[-20000:] # subsample for model selection experiments

tX1, tY1 = final_pipeline(tX0, tY0)
vX1, vY1 = final_pipeline(vX0, vY0)
vX1p = final_pipeline(vX)

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


In [ ]:
# %%time
# from sklearn.metrics import classification_report
# from sklearn.model_selection import cross_val_predict

# tX0, tY0 = tX.iloc[:20000,:], tY[:20000]   # subsample for model selection experiments
# vX0, vY0 = tX.iloc[-20000:,:], tY[-20000:] # subsample for model selection experiments

# tX1, tY1 = final_pipeline(tX0,tY0)
# vX1, vY1 = final_pipeline(vX0,vY0)
# vX1p = final_pipeline(vX)

poly = PolynomialFeatures(degree=4)                                 # add non-linear features (powers and interactions)
tX0p = poly.fit_transform(tX1.select_dtypes(include=np.number))     # create object on training set
vX0p = poly.transform(vX1.select_dtypes(include=np.number))         # apply the same object to test set
vXp  = poly.transform(vX1p.select_dtypes(include=np.number))        # apply the same object to test set

m = QDA()
#m1= LDA()

#parameters = {'reg_param':[0.1, 0.9]}
#m=GridSearchCV(model,parameters, cv=5)
#m2 = LR(C=.0001, max_iter=100, n_jobs=-1, multi_class='auto', random_state=0) # always seed your experiments for reproducibility


####print(m.fit(tX0p, tY0).score(vX0p, vY0))   # fit the model and show test error

score = m.fit(tX0p, tY0).score(vX0p, vY0)
#score = m.fit(tX1, tY1).score(vX1, vY0)


print("score: ", score)
pred = cross_val_predict(estimator = m, X = tX0p, y = tY0, cv = 3)
print("Classification Report: \n", classification_report(tY0, pred))

#print("LDA:")
#print(m1.fit(tX0p, tY0).score(vX0p, vY0))
#print("LR:")
#print(m2.fit(tX0p, tY0).score(vX0p, vY0))

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


score:  0.94965


/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Classification Report: 
               precision    recall  f1-score   support

           G       0.90      0.96      0.93      6481
           Q       0.95      0.90      0.92      6385
           S       0.99      0.99      0.99      7134

    accuracy                           0.95     20000
   macro avg       0.95      0.95      0.95     20000
weighted avg       0.95      0.95      0.95     20000



In [ ]:
pY = pd.DataFrame(m.predict(vXp), index=range(1, len(vX)+1), columns=['Class'])  # ensure that labels and observations are in corresponding order
#pY = pd.DataFrame(m.predict(vX1p), index=range(1, len(vX1p)+1), columns=['Class']) 

#df.Class.fillna('unknown').value_counts()   # distribution of all train labels
#pY.value_counts()                           # distribution of predicted labels
ToCSV(pY, 'MySubmission')

# **References:**

1. Remember to cite your sources!


<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 4 sec


## 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest missclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support, or can the pattern be linearized via transformations?
1. How are date/categorical features treated by the model? Is there a [better way](https://www.google.com/search?q=ways+to+encode+categorical+data) to encode these (perhaps, ordinal) features? 
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?
1. Learn astronomy domain and features: [🎦](https://www.youtube.com/results?search_query=Quasi-star), [quasi-star](https://en.wikipedia.org/wiki/Quasi-star), [star](https://en.wikipedia.org/wiki/Star), [galaxy](https://en.wikipedia.org/wiki/Galaxy), [📃](https://arxiv.org/abs/2112.02026)
